# Medical Cost Analysis Project

Data set definition ??? 
Medical Cost Personal Datasets
 (<a href="https://www.kaggle.com/datasets/mirichoi0218/insurance">Medical Cost Personal Datasets
</a>)

Group Name ? 


## Load Libraries

In [28]:
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from numpy import sqrt
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

## Load Dataset

In [29]:
dataset = pd.read_csv('insurance.csv')
dataset.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Exploratory Data Analysis

In [30]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [31]:
dataset.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [32]:
for column_name in dataset.columns:
    print ("---- %s ---" % column_name)
    print (dataset[column_name].value_counts())

---- age ---
18    69
19    68
50    29
51    29
47    29
46    29
45    29
20    29
48    29
52    29
22    28
49    28
54    28
53    28
21    28
26    28
24    28
25    28
28    28
27    28
23    28
43    27
29    27
30    27
41    27
42    27
44    27
31    27
40    27
32    26
33    26
56    26
34    26
55    26
57    26
37    25
59    25
58    25
36    25
38    25
35    25
39    25
61    23
60    23
63    23
62    23
64    22
Name: age, dtype: int64
---- sex ---
male      676
female    662
Name: sex, dtype: int64
---- bmi ---
32.300    13
28.310     9
30.495     8
30.875     8
31.350     8
          ..
46.200     1
23.800     1
44.770     1
32.120     1
30.970     1
Name: bmi, Length: 548, dtype: int64
---- children ---
0    574
1    324
2    240
3    157
4     25
5     18
Name: children, dtype: int64
---- smoker ---
no     1064
yes     274
Name: smoker, dtype: int64
---- region ---
southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64
---

In [33]:
correlations = dataset.corr()
correlations

,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


In [34]:
charges_correlations = correlations["charges"]
charges_correlations

age         0.299008
bmi         0.198341
children    0.067998
charges     1.000000
Name: charges, dtype: float64

In [35]:
region_groupby = dataset.groupby(by=["region"])['children'].agg(['count'])
region_groupby

,count
region,
northeast,324
northwest,325
southeast,364
southwest,325


In [36]:
res = region_groupby.apply(lambda x: x.sort_values(ascending=False).head(1))
res

,count
region,
southeast,364


### Data Visualization

In [10]:
#TODO: Data visualization
#Abdurrahman Keskin

## Data Preprocessing

### Converting objects labels into categorical

In [11]:
#Converting objects labels into categorical
dataset[['sex', 'smoker', 'region']] = dataset[['sex', 'smoker', 'region']].astype('category')
dataset.dtypes

age            int64
sex         category
bmi          float64
children       int64
smoker      category
region      category
charges      float64
dtype: object

In [37]:
dataset = pd.get_dummies(dataset, columns = ['region'])
dataset

#TODO: sex and smoker label encoder, region one hot encoder

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,female,27.900,0,yes,16884.92400,0,0,0,1
1,18,male,33.770,1,no,1725.55230,0,0,1,0
2,28,male,33.000,3,no,4449.46200,0,0,1,0
3,33,male,22.705,0,no,21984.47061,0,1,0,0
4,32,male,28.880,0,no,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830,0,1,0,0
1334,18,female,31.920,0,no,2205.98080,1,0,0,0
1335,18,female,36.850,0,no,1629.83350,0,0,1,0
1336,21,female,25.800,0,no,2007.94500,0,0,0,1


In [38]:
labelencoder = LabelEncoder()
dataset['sex'] = labelencoder.fit_transform(dataset['sex'].values)
dataset['smoker'] = labelencoder.fit_transform(dataset['smoker'].values)
dataset.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


### Normalize data

In [13]:
#TODO: Esra

### Splitting data into training and test data

In [39]:
train, test = train_test_split(dataset, test_size=0.3)

In [40]:
X_train = train.drop('charges', axis=1)
y_train = train.loc[:, 'charges']

In [41]:
X_test = test.drop('charges', axis=1)
y_test = test.loc[:, 'charges']

In [42]:
type(X_train)

pandas.core.frame.DataFrame

In [43]:
type(y_test)

pandas.core.series.Series

## Modelling


In [44]:
linear_regression_model = LinearRegression()
ridge_regression_model = Ridge(alpha=0.05, normalize=True)
#TODO: Lasso, SVR, DecisionTreeRegressor, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
# Baha

In [45]:
linear_regression_model.fit(X_train, y_train)
ridge_regression_model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  FutureWarning,


Ridge(alpha=0.05, normalize=True)

## Prediction

In [46]:
linear_regression_prediction = linear_regression_model.predict(X_test)
ridge_regression_prediction = ridge_regression_model.predict(X_test)

## Model Evaluation

In [47]:
mse_linear_regression = mean_squared_error(y_test, linear_regression_prediction)
mse_linear_regression

33625484.76805505

In [48]:
rmse_linear_regression = sqrt(mean_squared_error(y_test, linear_regression_prediction))
rmse_linear_regression

5798.748551890749

In [49]:
mse_ridge_regression = mean_squared_error(y_test, ridge_regression_prediction)
mse_ridge_regression

33829374.55014027

In [50]:
rmse_ridge_regression = sqrt(mean_squared_error(y_test, ridge_regression_prediction))
rmse_ridge_regression

5816.302480970214

In [51]:
dataset_prediction = pd.DataFrame({"charges" : y_test, "charges_prediction" : linear_regression_prediction})
dataset_prediction.head(20)

,charges,charges_prediction
1082,5855.90250,4688.851261
1306,16115.30450,27231.959093
428,3167.45585,-54.887765
700,2020.55230,4419.709598
265,46151.12450,38213.440669
254,41097.16175,35830.465884
602,11070.53500,10133.485357
439,2897.32350,4999.587560
1033,13747.87235,24223.563948
247,1986.93340,5406.118966


## Hyperparameter optimization

In [27]:
#TODO: hyperparameter optimization
# Label encoder, hyperparameter -> Melisa